The complete procedure of ion shuttling simulation is as follows:  
1. Find a target potential moving function, which is usually represented by the motion of the minimum point of the potential, i.e., $x_0 (t)$.  
2. Fit the endcap voltages as functions of the position of the minimum potential point ($V_{endcap}(x_0)$).  
3. Plug in $x_0(t)$, we can obtain $V_{endcap}(t)$.  
4. Using $V_{endcap}(t)$, we can find $\omega(t)$ and $x_0(t)$.  
5. Using $\omega(t)$ and $x_0(t)$, we can define the equations of motion of the ions and solve for the ions' motion.

\* When we are trying to find the target potential moving function, we can set $\omega$ to be constant and solve an ideal ode to see the performance of a certain moving function (no fitting needed). This is a crude but efficient method.

Since polynomial fit is frequently used during this procedure, I put the fitting function first:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
def pfit(x_data,y_data,degree,plot):
    coeffs=np.polyfit(x_data,y_data,degree)
    poly=np.poly1d(coeffs)
    y_fit=poly(x_data)
    if plot == True:
        plt.plot(x_data,y_data,label='original')
        plt.plot(x_data,y_fit,label='fit')
        plt.legend()
        plt.show()
    return coeffs

Usually, we fit the electric potential as a harmonic potential, which can be done by the following functions:

In [ ]:
def mini(x_data,y_data): #find the minimum point of the potential
    min_index=np.argmin(y_data)
    return x_data[min_index]

In [ ]:
def v_har(delta_x,x_data,y_data,plot): #delta_x set the region that we want to use for fitting
    degree=2
    fit_x_data=[]
    fit_y_data=[]
    for j in x_data:
        if mini(x_data,y_data)-delta_x<=j and j<=mini(x_data,y_data)+delta_x:
            fit_x_data.append(j)
            fit_y_data.append(y_data[x_data.tolist().index(j)])
    coeffs=np.polyfit(fit_x_data,fit_y_data,degree)
    poly=np.poly1d(coeffs)
    y_fit=poly(x_data)
    if plot == True:
        plt.plot(x_data,y_data,label='original')
        plt.plot(x_data,y_fit,label='fit')
        plt.xlabel('x/m')
        plt.ylabel('potential energy/J')
        plt.legend()
        plt.show()
    return coeffs

To calculate the electric potential given endcap voltages, I used the control voltage method, i.e., the electric potential due to $V_{endcap1}$ and $V_{endcap2}$ is the linear combination of the electric potential due to $V_{endcap1} = 1V$ $V_{endcap2} = 0V$ and $V_{endcap1} = 0V$ $V_{endcap2} = 1V$. The following function can find angular trap frequency and minimum potential point given 2 endcap voltages. Note that the data of $V_{endcap1} = 1V$ $V_{endcap2} = 0V$ and $V_{endcap1} = 0V$ $V_{endcap2} = 1V$ should be calculated using comsol and stored in the same folder as this program.

In [ ]:
def ctrl_vol(DC1,DC2,delta_x,plot):
    data_dc1 = np.loadtxt('DC1.csv',delimiter=',',skiprows=9)
    data_dc2 = np.loadtxt('DC2.csv',delimiter=',',skiprows=9)
    V1 = 1.6e-19*DC1 * data_dc1[:,3]
    V2 = 1.6e-19*DC2 * data_dc2[:,3]
    V = V1 + V2
    z = 1e-3*data_dc1[:,2] #note that the unit of the spatial coordinate should be m
    a, b, c = v_har(delta_x, z, V, plot)
    omega = math.sqrt(2*a/m)
    x_0 = -b/(m*omega**2) #this x0 should be more accurate than the one obtained by mini()
    return  omega, x_0, c

For our convenience, the function below can take $V_{endcap}(t)$ as input and calculate $V_{endcap1}$ and $V_{endcap2}$ of the sample points that we take for a transportation.

In [ ]:
from sympy.utilities.lambdify import lambdify
from sympy import sympify
from sympy import var
def cal_v():
    V = [[] for i in range(3)]
    t0 = float(input("Please input the start time: "))
    tf = float(input("Please input the stop time: "))
    dt = float(input("Please input the step size: "))
    for i in range(2):
        x = var('t')
        f = input(f"Please input the voltage function of DC {i+1} (using t as the variable): ")
        func = lambdify(x, sympify(f))
        t = t0
        while(t <= tf):
            if i == 0:
                V[0].append(t)
            V[i+1].append(func(t))
            t += dt
    return np.array(V) #V[t[], DC1[], DC2[]]


Then, we can find $\omega (t)$ and $x_0(t)$ given $V_{endcap}(t)$:

In [ ]:
arr = cal_v()
arr_new = [[] for i in range(3)]
index = 0
order = 5 #define the order of the polynomial fit
for i in arr[0]:
    arr_new[0].append(i)
    omega, x0, c = ctrl_vol(arr[1][index],arr[2][index],1e-4,False) 
    arr_new[1].append(omega)
    arr_new[2].append(x0)
    index += 1
a1,a2,a3,a4,a5,a6 = pfit(arr_new[0],arr_new[1],order,True) #fit omega_t
print(f'omega_t = {a1}*t**5+{a2}*t**4+{a3}*t**3+{a4}*t**2+{a5}*t+{a6}')
b1,b2,b3,b4,b5,b6 = pfit(arr_new[0],arr_new[2],order,True) #fit x0_t
print(f'x0_t = {b1}*t**5+{b2}*t**4+{b3}*t**3+{b4}*t**2+{b5}*t+{b6}')


The last step is to solve the ode. There are two versions of ode, one is for 1 step transportation (only one $\omega(t)$ and $x_0(t)$), while the other one is for 8 steps transportation ($\omega(t)$ and $x_0(t)$ are piecewise functions).

In [ ]:
def ode_9ions(t,y,t0,tf,omega_t,x0_t):
    x1,x2,x3,x4,x5,x6,x7,x8,x9,v1,v2,v3,v4,v5,v6,v7,v8,v9=y
    x = var('t')
    omega_t = lambdify(x, sympify(omega_t))
    x0_t = lambdify(x, sympify(x0_t))
    if t0 <= t and t <= tf:
        omega = omega_t(t)
        x0 = x0_t(t)
    elif t > tf:
        omega = omega_t(tf)
        x0 = x0_t(tf)
    a=-m*omega**2
    b=m*omega**2*x0
    dx1_dt=v1
    dx2_dt=v2
    dx3_dt=v3
    dx4_dt=v4
    dx5_dt=v5
    dx6_dt=v6
    dx7_dt=v7
    dx8_dt=v8
    dx9_dt=v9
    dv1_dt=1/m*(a*x1+b-e**2/(4*pi*E0)*(1/(x1-x2)**2+1/(x1-x3)**2+1/(x1-x4)**2+1/(x1-x5)**2+1/(x1-x6)**2+1/(x1-x7)**2+1/(x1-x8)**2+1/(x1-x9)**2))
    dv2_dt=1/m*(a*x2+b-e**2/(4*pi*E0)*(-1/(x2-x1)**2+1/(x2-x3)**2+1/(x2-x4)**2+1/(x2-x5)**2+1/(x2-x6)**2+1/(x2-x7)**2+1/(x2-x8)**2+1/(x2-x9)**2))
    dv3_dt=1/m*(a*x3+b-e**2/(4*pi*E0)*(-1/(x3-x1)**2-1/(x3-x2)**2+1/(x3-x4)**2+1/(x3-x5)**2+1/(x3-x6)**2+1/(x3-x7)**2+1/(x3-x8)**2+1/(x3-x9)**2))
    dv4_dt=1/m*(a*x4+b-e**2/(4*pi*E0)*(-1/(x4-x1)**2-1/(x4-x2)**2-1/(x4-x3)**2+1/(x4-x5)**2+1/(x4-x6)**2+1/(x4-x7)**2+1/(x4-x8)**2+1/(x4-x9)**2))
    dv5_dt=1/m*(a*x5+b-e**2/(4*pi*E0)*(-1/(x5-x1)**2-1/(x5-x2)**2-1/(x5-x3)**2-1/(x5-x4)**2+1/(x5-x6)**2+1/(x5-x7)**2+1/(x5-x8)**2+1/(x5-x9)**2))
    dv6_dt=1/m*(a*x6+b-e**2/(4*pi*E0)*(-1/(x6-x1)**2-1/(x6-x2)**2-1/(x6-x3)**2-1/(x6-x4)**2-1/(x6-x5)**2+1/(x6-x7)**2+1/(x6-x8)**2+1/(x6-x9)**2))
    dv7_dt=1/m*(a*x7+b-e**2/(4*pi*E0)*(-1/(x7-x1)**2-1/(x7-x2)**2-1/(x7-x3)**2-1/(x7-x4)**2-1/(x7-x5)**2-1/(x7-x6)**2+1/(x7-x8)**2+1/(x7-x9)**2))
    dv8_dt=1/m*(a*x8+b-e**2/(4*pi*E0)*(-1/(x8-x1)**2-1/(x8-x2)**2-1/(x8-x3)**2-1/(x8-x4)**2-1/(x8-x5)**2-1/(x8-x6)**2-1/(x8-x7)**2+1/(x8-x9)**2))
    dv9_dt=1/m*(a*x9+b-e**2/(4*pi*E0)*(-1/(x9-x1)**2-1/(x9-x2)**2-1/(x9-x3)**2-1/(x9-x4)**2-1/(x9-x5)**2-1/(x9-x6)**2-1/(x9-x7)**2-1/(x9-x8)**2))
    return [dx1_dt,dx2_dt,dx3_dt,dx4_dt,dx5_dt,dx6_dt,dx7_dt,dx8_dt,dx9_dt,dv1_dt,dv2_dt,dv3_dt,dv4_dt,dv5_dt,dv6_dt,dv7_dt,dv8_dt,dv9_dt]


In [ ]:
def ode_step8(t,y,T,omega,x0):
    x1,x2,x3,x4,x5,x6,x7,x8,x9,v1,v2,v3,v4,v5,v6,v7,v8,v9=y
    omega1,omega2,omega3,omega4,omega5,omega6,omega7,omega8 = omega
    t0,t1,t20,t2,t30,t3,t40,t4,t50,t5,t60,t6,t70,t7,t80,t8 = T
    x0_1,x0_2,x0_3,x0_4,x0_5,x0_6,x0_7,x0_8 = x0
    x = var('t')
    if t0 <= t <= t1:
        omega_t = lambdify(x, sympify(omega1))
        x0_t = lambdify(x, sympify(x0_1))
        omega_value = omega_t(t)
        x0_value = x0_t(t)
    elif t1 <= t <= t20:
        omega_t = lambdify(x, sympify(omega1))
        x0_t = lambdify(x, sympify(x0_1))
        omega_value = omega_t(t1)
        x0_value = x0_t(t1)
    elif t20 <= t <= t2:
        t_adjusted = t-t20
        omega_t = lambdify(x, sympify(omega2))
        x0_t = lambdify(x, sympify(x0_2))
        omega_value = omega_t(t_adjusted)
        x0_value = x0_t(t_adjusted)
    elif t2 <= t <= t30:
        t_adjusted = t2-t20
        omega_t = lambdify(x, sympify(omega2))
        x0_t = lambdify(x, sympify(x0_2))
        omega_value = omega_t(t_adjusted)
        x0_value = x0_t(t_adjusted)
    elif t30 <= t <= t3:
        t_adjusted = t-t30
        omega_t = lambdify(x, sympify(omega3))
        x0_t = lambdify(x, sympify(x0_3))
        omega_value = omega_t(t_adjusted)
        x0_value = x0_t(t_adjusted)
    elif t3 <= t <= t40:
        t_adjusted = t3-t30
        omega_t = lambdify(x, sympify(omega3))
        x0_t = lambdify(x, sympify(x0_3))
        omega_value = omega_t(t_adjusted)
        x0_value = x0_t(t_adjusted)
    elif t40 <= t <= t4:
        t_adjusted = t-t40
        omega_t = lambdify(x, sympify(omega4))
        x0_t = lambdify(x, sympify(x0_4))
        omega_value = omega_t(t_adjusted)
        x0_value = x0_t(t_adjusted)
    elif t4 <= t <= t50:
        t_adjusted = t4-t40
        omega_t = lambdify(x, sympify(omega4))
        x0_t = lambdify(x, sympify(x0_4))
        omega_value = omega_t(t_adjusted)
        x0_value = x0_t(t_adjusted)
    elif t50 <= t <=t5:
        t_adjusted = t-t50
        omega_t = lambdify(x, sympify(omega5))
        x0_t = lambdify(x, sympify(x0_5))
        omega_value = omega_t(t_adjusted)
        x0_value = x0_t(t_adjusted)
    elif t5 <= t <= t60:
        t_adjusted = t5-t50
        omega_t = lambdify(x, sympify(omega5))
        x0_t = lambdify(x, sympify(x0_5))
        omega_value = omega_t(t_adjusted)
        x0_value = x0_t(t_adjusted)
    elif t60 <= t <= t6:
        t_adjusted = t-t60
        omega_t = lambdify(x, sympify(omega6))
        x0_t = lambdify(x, sympify(x0_6))
        omega_value = omega_t(t_adjusted)
        x0_value = x0_t(t_adjusted)
    elif t6 <= t <= t70:
        t_adjusted = t6-t60
        omega_t = lambdify(x, sympify(omega6))
        x0_t = lambdify(x, sympify(x0_6))
        omega_value = omega_t(t_adjusted)
        x0_value = x0_t(t_adjusted)
    elif t70 <= t <= t7:
        t_adjusted = t-t70
        omega_t = lambdify(x, sympify(omega7))
        x0_t = lambdify(x, sympify(x0_7))
        omega_value = omega_t(t_adjusted)
        x0_value = x0_t(t_adjusted)
    elif t7 <= t <= t80:
        t_adjusted = t7-t70
        omega_t = lambdify(x, sympify(omega7))
        x0_t = lambdify(x, sympify(x0_7))
        omega_value = omega_t(t_adjusted)
        x0_value = x0_t(t_adjusted)
    elif t80 <= t <= t8:
        t_adjusted = t-t80
        omega_t = lambdify(x, sympify(omega8))
        x0_t = lambdify(x, sympify(x0_8))
        omega_value = omega_t(t_adjusted)
        x0_value = x0_t(t_adjusted)
    elif t > t8:
        t_adjusted = t8-t80
        omega_t = lambdify(x, sympify(omega8))
        x0_t = lambdify(x, sympify(x0_8))
        omega_value = omega_t(t_adjusted)
        x0_value = x0_t(t_adjusted)
    a=-m*omega_value**2
    b=m*omega_value**2*x0_value
    dx1_dt=v1
    dx2_dt=v2
    dx3_dt=v3
    dx4_dt=v4
    dx5_dt=v5
    dx6_dt=v6
    dx7_dt=v7
    dx8_dt=v8
    dx9_dt=v9
    dv1_dt=1/m*(a*x1+b-e**2/(4*pi*E0)*(1/(x1-x2)**2+1/(x1-x3)**2+1/(x1-x4)**2+1/(x1-x5)**2+1/(x1-x6)**2+1/(x1-x7)**2+1/(x1-x8)**2+1/(x1-x9)**2))
    dv2_dt=1/m*(a*x2+b-e**2/(4*pi*E0)*(-1/(x2-x1)**2+1/(x2-x3)**2+1/(x2-x4)**2+1/(x2-x5)**2+1/(x2-x6)**2+1/(x2-x7)**2+1/(x2-x8)**2+1/(x2-x9)**2))
    dv3_dt=1/m*(a*x3+b-e**2/(4*pi*E0)*(-1/(x3-x1)**2-1/(x3-x2)**2+1/(x3-x4)**2+1/(x3-x5)**2+1/(x3-x6)**2+1/(x3-x7)**2+1/(x3-x8)**2+1/(x3-x9)**2))
    dv4_dt=1/m*(a*x4+b-e**2/(4*pi*E0)*(-1/(x4-x1)**2-1/(x4-x2)**2-1/(x4-x3)**2+1/(x4-x5)**2+1/(x4-x6)**2+1/(x4-x7)**2+1/(x4-x8)**2+1/(x4-x9)**2))
    dv5_dt=1/m*(a*x5+b-e**2/(4*pi*E0)*(-1/(x5-x1)**2-1/(x5-x2)**2-1/(x5-x3)**2-1/(x5-x4)**2+1/(x5-x6)**2+1/(x5-x7)**2+1/(x5-x8)**2+1/(x5-x9)**2))
    dv6_dt=1/m*(a*x6+b-e**2/(4*pi*E0)*(-1/(x6-x1)**2-1/(x6-x2)**2-1/(x6-x3)**2-1/(x6-x4)**2-1/(x6-x5)**2+1/(x6-x7)**2+1/(x6-x8)**2+1/(x6-x9)**2))
    dv7_dt=1/m*(a*x7+b-e**2/(4*pi*E0)*(-1/(x7-x1)**2-1/(x7-x2)**2-1/(x7-x3)**2-1/(x7-x4)**2-1/(x7-x5)**2-1/(x7-x6)**2+1/(x7-x8)**2+1/(x7-x9)**2))
    dv8_dt=1/m*(a*x8+b-e**2/(4*pi*E0)*(-1/(x8-x1)**2-1/(x8-x2)**2-1/(x8-x3)**2-1/(x8-x4)**2-1/(x8-x5)**2-1/(x8-x6)**2-1/(x8-x7)**2+1/(x8-x9)**2))
    dv9_dt=1/m*(a*x9+b-e**2/(4*pi*E0)*(-1/(x9-x1)**2-1/(x9-x2)**2-1/(x9-x3)**2-1/(x9-x4)**2-1/(x9-x5)**2-1/(x9-x6)**2-1/(x9-x7)**2-1/(x9-x8)**2))
    return [dx1_dt,dx2_dt,dx3_dt,dx4_dt,dx5_dt,dx6_dt,dx7_dt,dx8_dt,dx9_dt,dv1_dt,dv2_dt,dv3_dt,dv4_dt,dv5_dt,dv6_dt,dv7_dt,dv8_dt,dv9_dt]
